<a href="https://colab.research.google.com/github/hardik4555/business-case-Deep-neural-networks-/blob/main/business_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import data**

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing #using sklearn to standardize the inputs
from google.colab import files
uploaded=files.upload()


Saving Audiobooks_data.csv to Audiobooks_data (2).csv


In [ ]:
raw_csv_data=np.loadtxt('Audiobooks_data.csv', delimiter=',')
unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all=raw_csv_data[:,-1]
#we will keep as many 0s as their are 1s

**Balance the dataset**

In [ ]:
num_one_targets= int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]

for i in range(targets_all.shape[0]):
  if targets_all[i]==0:
    zero_targets_counter+=1
    if zero_targets_counter>num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors= np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors= np.delete(targets_all, indices_to_remove, axis=0)

**Standardize the inputs**

In [ ]:
scaled_inputs= preprocessing.scale(unscaled_inputs_equal_priors)

**Shuffle the data**

In [ ]:
#we must shuffle the data
shuffled_indices= np.arange(scaled_inputs.shape[0])
# print(shuffled_indices)
np.random.shuffle(shuffled_indices)
# print(shuffled_indices)
shuffled_inputs= scaled_inputs[shuffled_indices]
shuffled_targets= targets_equal_priors[shuffled_indices]

**Split the data into train, validation, test**

In [ ]:
samples_count=shuffled_inputs.shape[0]
#using 80-10-10 split
train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_count=int(0.1*samples_count)

train_inputs= shuffled_inputs[:train_samples_count]
train_targets= shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets=shuffled_targets[train_samples_count: train_samples_count+ validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:-1]
test_targets=shuffled_targets[train_samples_count+validation_samples_count:-1]

**Save the three datasets in *.npz**

In [ ]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

**Creating the machine learning algorithm**

In [ ]:
npz= np.load('Audiobooks_data_train.npz')
train_inputs=npz['inputs'].astype(float)
train_targets=npz['targets'].astype(int)

npz=np.load('Audiobooks_data_validation.npz')
validation_inputs=npz['inputs'].astype(float)
validation_targets=npz['targets'].astype(int)

npz=np.load('Audiobooks_data_test.npz')
test_inputs=npz['inputs'].astype(float)
test_targets=npz['targets'].astype(int)

In [ ]:
train_targets.shape,validation_targets.shape

((3579,), (447,))

**Model**

In [ ]:
import tensorflow as tf

input_size=10
output_size=2
hidden_layer_size=100

model=tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'), #as we want a probablity b/w 0 to 1
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
batch_size=100
max_epochs=100
early_stopping= tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=max_epochs, callbacks=early_stopping, validation_data=(validation_inputs, validation_targets), verbose=2)


Epoch 1/100
36/36 - 1s - loss: 0.5467 - accuracy: 0.7181 - val_loss: 0.4461 - val_accuracy: 0.7875 - 1s/epoch - 32ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4136 - accuracy: 0.7726 - val_loss: 0.3993 - val_accuracy: 0.7919 - 199ms/epoch - 6ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3798 - accuracy: 0.7932 - val_loss: 0.3901 - val_accuracy: 0.7785 - 162ms/epoch - 5ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3638 - accuracy: 0.8108 - val_loss: 0.3723 - val_accuracy: 0.7919 - 164ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3544 - accuracy: 0.8097 - val_loss: 0.3672 - val_accuracy: 0.7852 - 162ms/epoch - 4ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3456 - accuracy: 0.8148 - val_loss: 0.3569 - val_accuracy: 0.8098 - 162ms/epoch - 4ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3431 - accuracy: 0.8184 - val_loss: 0.3665 - val_accuracy: 0.7987 - 201ms/epoch - 6ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3445 - accuracy: 0.8083 - val_loss: 0.3698 - val_accuracy: 0.7942 - 176ms/epoch - 5ms/step


**Test the model**

In [ ]:
test_loss, test_accuracy=model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3409 - accuracy: 0.8166


In [ ]:
print('\nTest loss: {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100,))


Test loss: 0.34, Test accuracy: 81.66%
